In [1]:


import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests



In [5]:
!pip install wikipedia

  Running setup.py bdist_wheel for wikipedia ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.


In [6]:
#wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#raw_wiki= requests.get(wiki_url).text
#soup = BeautifulSoup(raw_wiki,'html5lib')

In [7]:
#table=soup.find('table')

In [8]:
# other variant
import wikipedia as wp

html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.head()

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


In [9]:
df.drop(0,inplace= True)

In [10]:
df = df.rename(columns={0:'PostalCode', 1:'Borough' ,2:'Neighbourhood'})

df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [11]:
df = df.reset_index(drop=True)


In [12]:
for i in range(0,len(df)):
 if df['Borough'][i] == 'Not assigned':
  df.drop(i,inplace=True)

In [13]:
final_dataframe = pd.DataFrame(df).reset_index(drop=True)

In [14]:
final_dataframe = final_dataframe.rename( columns ={0:'PostalCode' , 1:'Borough' ,2:'Neighbourhood'} )

In [15]:
location= pd.read_csv('http://cocl.us/Geospatial_data')




In [16]:
location.rename(columns={'Postal Code' : 'PostalCode'} ,inplace=True)

In [17]:
final_dataframe = pd.merge(final_dataframe, location, on='PostalCode' ,how ='inner')

In [18]:
location.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:

final_dataframe.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [20]:
final_dataframe.drop_duplicates(subset=['PostalCode'] ,keep='first' ,inplace=True)
final_dataframe.drop( 'PostalCode',axis=1,inplace=True)

In [21]:
toronto = final_dataframe[final_dataframe["Borough"].str.contains('Toronto')]

In [22]:
toronto.head().reset_index(drop=True)

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,Downtown Toronto,Ryerson,43.657162,-79.378937
2,Downtown Toronto,St. James Town,43.651494,-79.375418
3,East Toronto,The Beaches,43.676357,-79.293031
4,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [23]:
from geopy.geocoders import Nominatim
locator = Nominatim()
location = locator.geocode('Toronto')
toronto_latitude = location.latitude
toronto_longitude = location.longitude





In [24]:
import folium
# define the world map centered around Canada with a higher zoom level
centralToronto_map = folium.Map(location=[43.715383, -79.405678], zoom_start=8)

# display  map
centralToronto_map

In [46]:



toronto_map  = folium.Map(location=[43.715383, -79.405678], zoom_start=11)

for lat ,lon ,borough ,neighborhood in zip( toronto['Latitude'] ,toronto['Longitude'] ,toronto['Borough'],toronto['Neighbourhood'] ):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lon], radius=7, popup=label, color='black', fill=True, fill_color='##3186cc', fill_opacity=0.5)\
    .add_to(toronto_map)

toronto_map

In [26]:
CLIENT_ID = 'WXA2H4LKJEKPHUYE0NTWJ5UJQWFY4QBVYQV1FTATYR0LQXV3' # your Foursquare ID
CLIENT_SECRET = 'M1CULVBJUMGVO4S5IGBEQHQGSNH0UJ2HS5SEURHPZH1CR0TJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WXA2H4LKJEKPHUYE0NTWJ5UJQWFY4QBVYQV1FTATYR0LQXV3
CLIENT_SECRET:M1CULVBJUMGVO4S5IGBEQHQGSNH0UJ2HS5SEURHPZH1CR0TJ


In [27]:
import requests
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant in'formation for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:

toronto_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude']
                                  )

Harbourfront
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide
Dovercourt Village
Harbourfront East
Little Portugal
The Danforth West
Design Exchange
Brockton
The Beaches West
Commerce Court
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North
High Park
North Toronto West
The Annex
Parkdale
Davisville
Harbord
Runnymede
Moore Park
Chinatown
Deer Park
CN Tower
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown
First Canadian Place
Church and Wellesley
Business Reply Mail Processing Centre 969 Eastern


In [29]:
toronto_venues

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.654260,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.654260,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.654260,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
5,Harbourfront,43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,Harbourfront,43.654260,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
7,Harbourfront,43.654260,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
8,Harbourfront,43.654260,-79.360636,Corktown Common,43.655618,-79.356211,Park
9,Harbourfront,43.654260,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


In [30]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Harbourfront,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_onehot.shape

(1701, 240)

In [32]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Adelaide,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.03,0.0,...,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.000000
1,Berczy Park,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0,0.000000
2,Brockton,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.052632
4,CN Tower,0.0,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


In [33]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4                  Bar  0.04
5           Restaurant  0.03
6       Breakfast Spot  0.03
7       Cosmetics Shop  0.03
8  American Restaurant  0.03
9                Hotel  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2              Bakery  0.05
3      Farmers Market  0.04
4            Beer Bar  0.04
5  Seafood Restaurant  0.04
6         Cheese Shop  0.04
7                Café  0.04
8          Steakhouse  0.04
9    Greek Restaurant  0.02


----Brockton----
                    venue  freq
0          Breakfast Spot  0.09
1                    Café  0.09
2             Coffee Shop  0.09
3                     Bar  0.05
4                     Gym  0.05
5                 Stadium  0.05
6  Furniture / Home Store  0.05
7    Caribbean Restaurant  0.05
8               Nightclub  

                 venue  freq
0                 Park   0.4
1                Trail   0.2
2           Playground   0.2
3             Building   0.2
4  Monument / Landmark   0.0
5         Neighborhood   0.0
6          Music Venue   0.0
7          Music Store   0.0
8               Museum   0.0
9        Movie Theater   0.0


----Roselawn----
                             venue  freq
0                           Garden   1.0
1                Afghan Restaurant   0.0
2                      Men's Store   0.0
3        Middle Eastern Restaurant   0.0
4               Miscellaneous Shop   0.0
5       Modern European Restaurant   0.0
6  Molecular Gastronomy Restaurant   0.0
7              Monument / Landmark   0.0
8                    Movie Theater   0.0
9                           Museum   0.0


----Runnymede----
                 venue  freq
0     Sushi Restaurant  0.08
1          Coffee Shop  0.08
2          Pizza Place  0.08
3                 Café  0.08
4             Tea Room  0.05
5   Italian Resta

In [34]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Breakfast Spot,Gym,Hotel,Restaurant
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Café,Beer Bar,Gourmet Shop
2,Brockton,Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Burrito Place,Stadium,Caribbean Restaurant,Restaurant,Bar,Furniture / Home Store
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Spa,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Park,Comic Shop,Pizza Place
4,CN Tower,Airport Service,Airport Lounge,Airport Terminal,Boutique,Airport,Airport Food Court,Airport Gate,Bar,Harbor / Marina,Sculpture Garden
5,Cabbagetown,Coffee Shop,Café,Restaurant,Italian Restaurant,Pub,Bakery,Pizza Place,Yoga Studio,Park,Sandwich Place
6,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Sushi Restaurant,Bar,Chinese Restaurant,Spa
7,Chinatown,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Dumpling Restaurant,Vietnamese Restaurant,Mexican Restaurant,Bakery,Coffee Shop,Record Shop
8,Christie,Grocery Store,Café,Park,Baby Store,Diner,Convenience Store,Nightclub,Italian Restaurant,Coffee Shop,Restaurant
9,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Pub,Fast Food Restaurant,Bubble Tea Shop,Hotel,Café


In [36]:
from sklearn.cluster import KMeans
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [37]:
manhattan_merged = toronto

# add clustering labels
manhattan_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

manhattan_merged.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Dessert Shop,Chocolate Shop
13,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Ice Cream Shop,Plaza,Bubble Tea Shop,Tea Room
27,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Clothing Store,Gastropub,Breakfast Spot,Beer Bar,Cosmetics Shop
36,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Pub,Trail,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
37,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Café,Beer Bar,Gourmet Shop


In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[toronto_latitude, toronto_longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood'],toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [39]:
clusters = toronto

# add clustering labels
clusters['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
clusters = clusters.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

clusters.head().reset_index(drop=True)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,Harbourfront,43.654260,-79.360636,0,Coffee Shop,Bakery,Café,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Dessert Shop,Chocolate Shop
1,Downtown Toronto,Ryerson,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Ice Cream Shop,Plaza,Bubble Tea Shop,Tea Room
2,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Clothing Store,Gastropub,Breakfast Spot,Beer Bar,Cosmetics Shop
3,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Pub,Trail,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Café,Beer Bar,Gourmet Shop


# The first cluster can be visualized as:

In [40]:
clusters.loc[clusters['Cluster Labels'] == 0, clusters.columns[[1] + list(range(5, clusters.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Harbourfront,Coffee Shop,Bakery,Café,Park,Pub,Mexican Restaurant,Breakfast Spot,Restaurant,Dessert Shop,Chocolate Shop
13,Ryerson,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Fast Food Restaurant,Ice Cream Shop,Plaza,Bubble Tea Shop,Tea Room
27,St. James Town,Coffee Shop,Italian Restaurant,Café,Restaurant,Hotel,Clothing Store,Gastropub,Breakfast Spot,Beer Bar,Cosmetics Shop
36,The Beaches,Health Food Store,Neighborhood,Pub,Trail,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
37,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Steakhouse,Seafood Restaurant,Farmers Market,Café,Beer Bar,Gourmet Shop
41,Central Bay Street,Coffee Shop,Italian Restaurant,Ice Cream Shop,Sandwich Place,Burger Joint,Café,Sushi Restaurant,Bar,Chinese Restaurant,Spa
42,Christie,Grocery Store,Café,Park,Baby Store,Diner,Convenience Store,Nightclub,Italian Restaurant,Coffee Shop,Restaurant
49,Adelaide,Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Cosmetics Shop,Breakfast Spot,Gym,Hotel,Restaurant
52,Dovercourt Village,Bakery,Pharmacy,Supermarket,Gym / Fitness Center,Pet Store,Pizza Place,Music Venue,Discount Store,Café,Middle Eastern Restaurant
61,Harbourfront East,Coffee Shop,Hotel,Aquarium,Café,Italian Restaurant,Pizza Place,Brewery,Scenic Lookout,Sporting Goods Shop,Bakery


# The second cluster can be visualized as

In [41]:
clusters.loc[clusters['Cluster Labels'] == 1, clusters.columns[[1] + list(range(5, clusters.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
134,Parkdale,Gift Shop,Breakfast Spot,Bookstore,Movie Theater,Eastern European Restaurant,Bar,Bank,Italian Restaurant,Restaurant,Dog Run


# The third cluster can be visualized as

In [42]:
clusters.loc[clusters['Cluster Labels'] == 2, clusters.columns[[1] + list(range(5, clusters.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
150,Moore Park,Summer Camp,Yoga Studio,Flea Market,Fish & Chips Shop,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


# thank you for you attention :) !